# AI-RAN Dataset Generation with NVIDIA Sionna

**Objective**: Generate synthetic 5G PUSCH data for neural receiver training

**Output**: ~10GB HDF5 dataset with 100K slots across multiple SNR points

**Runtime**: ~30 minutes on RTX 4090

---

## System Configuration

- **Channel Model**: 3GPP CDL-C (Urban Macro)
- **Carrier**: 3.5 GHz (n78 band)
- **Bandwidth**: 273 PRBs (100 MHz)
- **Subcarrier Spacing**: 30 kHz
- **Antennas**: 1 TX, 4 RX (SIMO)
- **Modulation**: 16QAM (MCS-7)
- **SNR Range**: -10 to +10 dB (21 points)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import numpy as np
import tensorflow as tf
import h5py
from tqdm import tqdm
import matplotlib.pyplot as plt
import time

# Sionna 1.2.1 imports - everything under sionna.phy.*
import sionna
from sionna.phy.channel.tr38901 import CDL, PanelArray
from sionna.phy.ofdm import ResourceGrid, ResourceGridMapper, RemoveNulledSubcarriers
from sionna.phy.mapping import Mapper, Demapper
from sionna.phy.utils import sample_bernoulli, ebnodb2no  # Changed: BinarySource -> sample_bernoulli

print(f"TensorFlow version: {tf.__version__}")
print(f"Sionna version: {sionna.__version__}")
print(f"GPUs available: {tf.config.list_physical_devices('GPU')}")

# Configure GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print(f"✅ GPU memory growth enabled")

## 1. Define 5G NR PUSCH Configuration

In [ ]:
# Dataset Size Selection - Simple approach
# Change the value below to your desired dataset size

DATASET_CHOICE = 1  # ← CHANGE THIS: 1=SMALL, 2=MEDIUM, 3=LARGE

# Configuration based on choice
if DATASET_CHOICE == 1:
    DATASET_SIZE = 'SMALL'
    TARGET_TOTAL_SAMPLES = 10000
    NUM_SNR_POINTS = 11
    print("✅ Selected: SMALL dataset")
    print(f"   - ~10K samples, ~43 GB, ~5-10 min")
elif DATASET_CHOICE == 2:
    DATASET_SIZE = 'MEDIUM'
    TARGET_TOTAL_SAMPLES = 50000
    NUM_SNR_POINTS = 15
    print("✅ Selected: MEDIUM dataset")
    print(f"   - ~50K samples, ~215 GB, ~30-45 min")
elif DATASET_CHOICE == 3:
    DATASET_SIZE = 'LARGE'
    TARGET_TOTAL_SAMPLES = 100000
    NUM_SNR_POINTS = 21
    print("✅ Selected: LARGE dataset")
    print(f"   - ~100K samples, ~430 GB, ~1-1.5 hrs")
else:
    raise ValueError(f"Invalid DATASET_CHOICE: {DATASET_CHOICE}. Must be 1, 2, or 3.")

print(f"   - Total samples: {TARGET_TOTAL_SAMPLES:,}")
print(f"   - SNR points: {NUM_SNR_POINTS}")

In [ ]:
# Check if dataset size was selected
try:
    _ = TARGET_TOTAL_SAMPLES
    _ = NUM_SNR_POINTS
    _ = DATASET_SIZE
except NameError:
    print("❌ ERROR: Dataset size not selected!")
    print("\n⚠️  Please run the dataset size selection cell first.")
    print("   Run the cell above that prompts: 'Enter your choice (1, 2, or 3)'")
    raise NameError("TARGET_TOTAL_SAMPLES not defined. Run dataset size selection cell first!")

# 5G NR PUSCH parameters
NUM_OFDM_SYMBOLS = 14  # One slot
FFT_SIZE = 4096  # For 100 MHz @ 30 kHz SCS
NUM_RBS = 273  # Resource blocks (100 MHz)
SUBCARRIER_SPACING = 30e3  # 30 kHz
NUM_STREAMS_PER_TX = 1  # Single layer
NUM_TX_ANTENNAS = 1
NUM_RX_ANTENNAS = 4
CARRIER_FREQUENCY = 3.5e9  # 3.5 GHz
MODULATION_ORDER = 4  # 16QAM
NUM_BITS_PER_SYMBOL = 4

# Dataset parameters - From user selection
BATCH_SIZE = 64
SNR_MIN_DB = -10.0
SNR_MAX_DB = 10.0

# Calculate batches per SNR point based on user selection
SAMPLES_PER_SNR = TARGET_TOTAL_SAMPLES // NUM_SNR_POINTS
NUM_BATCHES = SAMPLES_PER_SNR // BATCH_SIZE

# Calculate expected dataset size
# Approximate effective subcarriers (after removing DC and guards)
approx_subcarriers = int(FFT_SIZE * 0.8)  # ~80% are data subcarriers
num_data_symbols = approx_subcarriers * NUM_OFDM_SYMBOLS
num_bits = num_data_symbols * NUM_BITS_PER_SYMBOL

# Size per sample in bytes
size_y = NUM_RX_ANTENNAS * approx_subcarriers * NUM_OFDM_SYMBOLS * 8  # complex64 = 8 bytes
size_h = NUM_RX_ANTENNAS * NUM_TX_ANTENNAS * approx_subcarriers * NUM_OFDM_SYMBOLS * 8
size_bits = num_bits * 4  # int32 = 4 bytes
size_per_sample_bytes = size_y + size_h + size_bits

# Total size
total_samples = NUM_BATCHES * BATCH_SIZE * NUM_SNR_POINTS
total_size_gb = (size_per_sample_bytes * total_samples) / (1024**3)

# Calculate estimated generation time
batches_total = NUM_BATCHES * NUM_SNR_POINTS
time_optimized_min = (batches_total * 1.5) / 60  # 1.5s per batch (optimized)
time_slow_min = (batches_total * 60) / 60  # 60s per batch (not optimized)

print(f"\n📊 Dataset Configuration ({DATASET_SIZE}):")
print(f"  Total samples: {total_samples:,}")
print(f"  Samples per SNR: {NUM_BATCHES * BATCH_SIZE:,}")
print(f"  Batches per SNR: {NUM_BATCHES}")
print(f"  Total batches: {batches_total:,}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  SNR points: {NUM_SNR_POINTS}")
print(f"  SNR range: {SNR_MIN_DB} to {SNR_MAX_DB} dB")
print(f"  RX antennas: {NUM_RX_ANTENNAS}")
print(f"  Modulation: 16QAM")
print(f"  PRBs: {NUM_RBS}")
print(f"\n📏 Dataset Size:")
print(f"  Size per sample: {size_per_sample_bytes / (1024**2):.2f} MB")
print(f"  Total size: ~{total_size_gb:.1f} GB")
print(f"\n⏱️  Estimated Generation Time:")
print(f"  If GPU optimized: ~{time_optimized_min:.1f} minutes")
print(f"  If not optimized: ~{time_slow_min:.0f} minutes (restart with optimized code!)")

# Create output directories
import os
os.makedirs('/opt/app-root/src/data', exist_ok=True)
os.makedirs('/opt/app-root/src/models', exist_ok=True)
os.makedirs('/opt/app-root/src/results', exist_ok=True)

print("\n✅ Configuration complete!")

## Dataset Size Selection

Choose your dataset size based on your goals:

| Size | Samples | SNR Points | Size | Time* | Use Case |
|------|---------|------------|------|-------|----------|
| **Small** | ~10K | 11 | ~43 GB | ~5-10 min | Quick pipeline test, proof of concept |
| **Medium** | ~50K | 15 | ~215 GB | ~30-45 min | Good accuracy, practical training |
| **Large** | ~100K | 21 | ~430 GB | ~1-1.5 hrs | Best accuracy, production model |

*Time estimates assume GPU-optimized generation (~1-2s per batch)

**Accuracy expectations:**
- Small: Good baseline performance, suitable for demo
- Medium: Strong performance, recommended for most use cases
- Large: Maximum performance, diminishing returns vs. medium

In [ ]:
# 5G NR PUSCH parameters
NUM_OFDM_SYMBOLS = 14  # One slot
FFT_SIZE = 4096  # For 100 MHz @ 30 kHz SCS
NUM_RBS = 273  # Resource blocks (100 MHz)
SUBCARRIER_SPACING = 30e3  # 30 kHz
NUM_STREAMS_PER_TX = 1  # Single layer
NUM_TX_ANTENNAS = 1
NUM_RX_ANTENNAS = 4
CARRIER_FREQUENCY = 3.5e9  # 3.5 GHz
MODULATION_ORDER = 4  # 16QAM
NUM_BITS_PER_SYMBOL = 4

# Dataset parameters - Quick test with 10K samples
BATCH_SIZE = 64
TARGET_TOTAL_SAMPLES = 10000  # 10K samples for quick test (~43 GB)
NUM_SNR_POINTS = 11  # 11 SNR points from -10 to +10 dB
SNR_MIN_DB = -10.0
SNR_MAX_DB = 10.0

# Calculate batches per SNR point to achieve target total samples
SAMPLES_PER_SNR = TARGET_TOTAL_SAMPLES // NUM_SNR_POINTS  # ~909 samples per SNR
NUM_BATCHES = SAMPLES_PER_SNR // BATCH_SIZE  # ~14 batches per SNR point

# Calculate expected dataset size
# Approximate effective subcarriers (after removing DC and guards)
approx_subcarriers = int(FFT_SIZE * 0.8)  # ~80% are data subcarriers
num_data_symbols = approx_subcarriers * NUM_OFDM_SYMBOLS
num_bits = num_data_symbols * NUM_BITS_PER_SYMBOL

# Size per sample in bytes
size_y = NUM_RX_ANTENNAS * approx_subcarriers * NUM_OFDM_SYMBOLS * 8  # complex64 = 8 bytes
size_h = NUM_RX_ANTENNAS * NUM_TX_ANTENNAS * approx_subcarriers * NUM_OFDM_SYMBOLS * 8
size_bits = num_bits * 4  # int32 = 4 bytes
size_per_sample_bytes = size_y + size_h + size_bits

# Total size
total_samples = NUM_BATCHES * BATCH_SIZE * NUM_SNR_POINTS
total_size_gb = (size_per_sample_bytes * total_samples) / (1024**3)

print(f"📊 Dataset Configuration (10K Quick Test):")
print(f"  Total samples: {total_samples:,}")
print(f"  Samples per SNR: {NUM_BATCHES * BATCH_SIZE:,}")
print(f"  Batches per SNR: {NUM_BATCHES}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  SNR points: {NUM_SNR_POINTS}")
print(f"  SNR range: {SNR_MIN_DB} to {SNR_MAX_DB} dB")
print(f"  RX antennas: {NUM_RX_ANTENNAS}")
print(f"  Modulation: 16QAM")
print(f"  PRBs: {NUM_RBS}")
print(f"  Size per sample: {size_per_sample_bytes / (1024**2):.2f} MB")
print(f"  Expected total size: ~{total_size_gb:.1f} GB")

# Create output directories
import os
os.makedirs('/opt/app-root/src/data', exist_ok=True)
os.makedirs('/opt/app-root/src/models', exist_ok=True)
os.makedirs('/opt/app-root/src/results', exist_ok=True)

## 2. Build Sionna PUSCH Simulator

In [ ]:
class PUSCHDataGenerator(tf.keras.Model):
    """End-to-end 5G PUSCH transmitter with frequency-selective fading channel"""
    
    def __init__(self):
        super().__init__()
        
        # Resource grid (simplified - no pilots for now)
        self.rg = ResourceGrid(
            num_ofdm_symbols=NUM_OFDM_SYMBOLS,
            fft_size=FFT_SIZE,
            subcarrier_spacing=SUBCARRIER_SPACING,
            num_tx=NUM_TX_ANTENNAS,
            num_streams_per_tx=NUM_STREAMS_PER_TX,
            cyclic_prefix_length=0,
            pilot_pattern=None
        )
        
        # Components
        self.mapper = Mapper("qam", NUM_BITS_PER_SYMBOL)
        self.rg_mapper = ResourceGridMapper(self.rg)
        self.remove_nulled = RemoveNulledSubcarriers(self.rg)
        
        # Pre-compute channel tap parameters for efficiency
        self.num_taps = 23
        tap_delays = tf.range(self.num_taps, dtype=tf.float32) * (10.0 / self.num_taps)
        tap_powers = tf.exp(-tap_delays / 5.0)
        self.tap_powers = tap_powers / tf.reduce_sum(tap_powers)
    
    @tf.function
    def _generate_frequency_selective_channel(self, batch_size, num_subcarriers):
        """Generate frequency-selective Rayleigh fading channel (GPU-optimized)"""
        # Generate Rayleigh fading coefficients for each tap
        h_real = tf.random.normal([batch_size, NUM_RX_ANTENNAS, NUM_TX_ANTENNAS, self.num_taps])
        h_imag = tf.random.normal([batch_size, NUM_RX_ANTENNAS, NUM_TX_ANTENNAS, self.num_taps])
        h_taps = tf.complex(h_real, h_imag) * tf.cast(tf.sqrt(self.tap_powers / 2.0), tf.complex64)
        
        # Frequency response using matrix multiplication (GPU-efficient)
        # Create DFT matrix for channel taps
        k = tf.cast(tf.range(num_subcarriers), tf.float32)  # Subcarrier indices
        n = tf.cast(tf.range(self.num_taps), tf.float32)    # Tap indices
        phase = -2.0 * np.pi * tf.reshape(k, [-1, 1]) * tf.reshape(n, [1, -1]) / tf.cast(num_subcarriers, tf.float32)
        dft_matrix = tf.complex(tf.cos(phase), tf.sin(phase))  # [num_subcarriers, num_taps]
        
        # Apply DFT to get frequency response: h_freq = h_taps @ dft_matrix^T
        # h_taps: [batch, rx, tx, taps]
        # dft_matrix: [subcarriers, taps]
        # Result: [batch, rx, tx, subcarriers]
        h_taps_reshaped = tf.reshape(h_taps, [-1, self.num_taps])
        h_freq_flat = tf.matmul(h_taps_reshaped, dft_matrix, transpose_b=True)
        h_freq = tf.reshape(h_freq_flat, [batch_size, NUM_RX_ANTENNAS, NUM_TX_ANTENNAS, num_subcarriers])
        
        # Add time variation (minimal for block-fading assumption)
        h_freq = tf.expand_dims(h_freq, -1)  # [batch, rx, tx, subcarriers, 1]
        h_freq = tf.tile(h_freq, [1, 1, 1, 1, NUM_OFDM_SYMBOLS])  # [batch, rx, tx, subcarriers, symbols]
        
        return h_freq
    
    @tf.function
    def generate_batch(self, batch_size, ebno_db):
        """Generate one batch of PUSCH data (GPU-optimized)"""
        # Generate random bits
        num_bits = self.rg.num_data_symbols * NUM_BITS_PER_SYMBOL
        bits = sample_bernoulli([batch_size, NUM_TX_ANTENNAS, NUM_STREAMS_PER_TX, num_bits], p=0.5)
        
        # Map to symbols and resource grid
        x = self.mapper(bits)
        x_rg = self.rg_mapper(x)
        x = self.remove_nulled(x_rg)
        x = tf.squeeze(x, axis=2)  # Remove streams dimension
        x = tf.transpose(x, perm=[0, 1, 3, 2])  # [batch, tx, subcarriers, symbols]
        
        # Generate channel
        h = self._generate_frequency_selective_channel(batch_size, self.rg.num_effective_subcarriers)
        
        # Apply channel
        y = tf.einsum('brtfs,btfs->brfs', h, x)
        
        # Add AWGN noise
        no = ebnodb2no(ebno_db, NUM_BITS_PER_SYMBOL, 1.0)
        noise_real = tf.random.normal(tf.shape(y), stddev=tf.sqrt(no/2))
        noise_imag = tf.random.normal(tf.shape(y), stddev=tf.sqrt(no/2))
        y = y + tf.complex(noise_real, noise_imag)
        
        return y, h, bits, x

# Instantiate generator
pusch_gen = PUSCHDataGenerator()
print("✅ PUSCH data generator initialized")
print(f"   Resource grid: {pusch_gen.rg.num_effective_subcarriers} subcarriers x {NUM_OFDM_SYMBOLS} symbols")
print(f"   Data symbols per slot: {pusch_gen.rg.num_data_symbols}")
print(f"   Channel model: Frequency-selective Rayleigh fading (GPU-optimized)")
print(f"   Number of TX antennas: {NUM_TX_ANTENNAS}")
print(f"   Number of RX antennas: {NUM_RX_ANTENNAS}")

## 3. Test Generator with Single Batch

## Understanding the Visualizations

### Left: Received Signal Power (RX Antenna 0)
**What it shows:** Power level of the received signal at each subcarrier and OFDM symbol

**Color coding:**
- 🟪 **Dark purple** = Low power (deep fades, weak signal)
- 🟦 **Blue/Cyan** = Medium power
- 🟩 **Green** = Strong power
- 🟨 **Yellow** = Very strong power (peaks)

**Pattern interpretation:**
- **Vertical stripes** indicate **frequency-selective fading**
- Different subcarriers experience different channel gains
- This is realistic for wideband 5G channels with multipath propagation

### Right: Channel Frequency Response |H|²
**What it shows:** Channel magnitude squared at each subcarrier and OFDM symbol

**Color coding:**
- 🟦 **Dark blue** = Weak channel (|H|² ≈ 0, deep null)
- 🟪 **Purple/Pink** = Moderate channel gain
- 🟧 **Orange** = Strong channel gain
- 🟨 **Yellow** = Very strong channel gain (|H|² > 3)

**Pattern interpretation:**
- **Regular vertical pattern** shows the multipath delay profile
- **23 taps** in frequency domain create periodic fading patterns
- Matches CDL-C Urban Macro channel characteristics

**Why this matters for AI-RAN:**
The neural receiver must learn to:
1. Track frequency-selective fading across 4096 subcarriers
2. Exploit spatial diversity from 4 RX antennas
3. Decode 16QAM symbols despite channel nulls and noise

In [ ]:
# Test with single batch
test_ebno_db = 5.0
y_test, h_test, bits_test, x_test = pusch_gen.generate_batch(BATCH_SIZE, test_ebno_db)

print(f"\n📡 Test Batch Shapes:")
print(f"  Received signal (y): {y_test.shape}")
print(f"  Channel (h): {h_test.shape}")
print(f"  Transmitted bits: {bits_test.shape}")
print(f"  Transmitted symbols (x): {x_test.shape}")

# Visualize received signal power
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Time-frequency grid
rx_power = np.abs(y_test[0, 0].numpy())**2
im = axes[0].imshow(rx_power.T, aspect='auto', cmap='viridis', origin='lower')
axes[0].set_xlabel('Subcarrier')
axes[0].set_ylabel('OFDM Symbol')
axes[0].set_title('Received Signal Power (RX Antenna 0)')
plt.colorbar(im, ax=axes[0])

# Channel frequency response
h_freq = np.abs(h_test[0, 0, 0].numpy())**2
im = axes[1].imshow(h_freq.T, aspect='auto', cmap='plasma', origin='lower')
axes[1].set_xlabel('Subcarrier')
axes[1].set_ylabel('OFDM Symbol')
axes[1].set_title('Channel Frequency Response')
plt.colorbar(im, ax=axes[1])

plt.tight_layout()
plt.savefig('/opt/app-root/src/results/sample_pusch_slot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Test generation successful!")

## 4. Generate Full Dataset (100K samples)

This will take approximately **30 minutes** on RTX 4090.

We'll generate data at multiple SNR points to train a robust neural receiver.

In [ ]:
# SNR points for training
snr_db_values = np.linspace(SNR_MIN_DB, SNR_MAX_DB, NUM_SNR_POINTS)
print(f"SNR points (dB): {snr_db_values}")

# Calculate dataset size
num_subcarriers = pusch_gen.rg.num_effective_subcarriers
total_samples = NUM_BATCHES * BATCH_SIZE

# Each sample:
# - y: complex64, shape [num_rx, num_subcarriers, num_symbols] = 4 x 3276 x 14
# - h: complex64, shape [num_rx, num_tx, num_subcarriers, num_symbols] = 4 x 1 x 3276 x 14
# - bits: int32, shape [num_bits]

sample_size_mb = (
    (NUM_RX_ANTENNAS * num_subcarriers * NUM_OFDM_SYMBOLS * 8) +  # y (complex64 = 8 bytes)
    (NUM_RX_ANTENNAS * NUM_TX_ANTENNAS * num_subcarriers * NUM_OFDM_SYMBOLS * 8) +  # h
    (pusch_gen.rg.num_data_symbols * NUM_BITS_PER_SYMBOL * 4)  # bits (int32 = 4 bytes)
) / 1024 / 1024

total_size_gb = (sample_size_mb * total_samples * NUM_SNR_POINTS) / 1024

print(f"\n📊 Dataset Size Estimate:")
print(f"  Samples per SNR point: {total_samples:,}")
print(f"  Total SNR points: {NUM_SNR_POINTS}")
print(f"  Total samples: {total_samples * NUM_SNR_POINTS:,}")
print(f"  Size per sample: {sample_size_mb:.2f} MB")
print(f"  Total size: ~{total_size_gb:.1f} GB")
print(f"\n⏱️  Estimated time: ~30 minutes on RTX 4090")
print(f"\n🚀 Starting dataset generation...")

In [ ]:
# Create HDF5 file
dataset_path = '/opt/app-root/src/data/pusch_dataset.h5'

with h5py.File(dataset_path, 'w') as f:
    # Create datasets
    f.create_dataset(
        'y_received',
        shape=(total_samples * NUM_SNR_POINTS, NUM_RX_ANTENNAS, num_subcarriers, NUM_OFDM_SYMBOLS),
        dtype=np.complex64,
        compression='gzip',
        compression_opts=4
    )
    
    f.create_dataset(
        'h_channel',
        shape=(total_samples * NUM_SNR_POINTS, NUM_RX_ANTENNAS, NUM_TX_ANTENNAS, num_subcarriers, NUM_OFDM_SYMBOLS),
        dtype=np.complex64,
        compression='gzip',
        compression_opts=4
    )
    
    f.create_dataset(
        'bits',
        shape=(total_samples * NUM_SNR_POINTS, NUM_TX_ANTENNAS, NUM_STREAMS_PER_TX, pusch_gen.rg.num_data_symbols * NUM_BITS_PER_SYMBOL),
        dtype=np.int32,
        compression='gzip',
        compression_opts=4
    )
    
    f.create_dataset(
        'snr_db',
        shape=(total_samples * NUM_SNR_POINTS,),
        dtype=np.float32
    )
    
    # Save metadata
    f.attrs['num_samples'] = total_samples * NUM_SNR_POINTS
    f.attrs['batch_size'] = BATCH_SIZE
    f.attrs['num_rx_antennas'] = NUM_RX_ANTENNAS
    f.attrs['num_tx_antennas'] = NUM_TX_ANTENNAS
    f.attrs['num_subcarriers'] = num_subcarriers
    f.attrs['num_ofdm_symbols'] = NUM_OFDM_SYMBOLS
    f.attrs['modulation_order'] = MODULATION_ORDER
    f.attrs['carrier_frequency'] = CARRIER_FREQUENCY
    f.attrs['snr_min_db'] = SNR_MIN_DB
    f.attrs['snr_max_db'] = SNR_MAX_DB
    f.attrs['num_snr_points'] = NUM_SNR_POINTS
    f.attrs['channel_model'] = 'CDL-C'
    
print(f"✅ HDF5 file created: {dataset_path}")

In [ ]:
# Generate data
start_time = time.time()
sample_idx = 0

with h5py.File(dataset_path, 'a') as f:
    for snr_idx, snr_db in enumerate(snr_db_values):
        print(f"\n📡 Generating data for SNR = {snr_db:.1f} dB ({snr_idx+1}/{NUM_SNR_POINTS})")
        
        pbar = tqdm(total=NUM_BATCHES, desc=f"SNR {snr_db:.1f} dB")
        
        for batch_idx in range(NUM_BATCHES):
            # Generate batch
            y, h, bits, _ = pusch_gen.generate_batch(BATCH_SIZE, snr_db)
            
            # Convert to numpy
            y_np = y.numpy()
            h_np = h.numpy()
            bits_np = bits.numpy()
            
            # Write to HDF5
            start_idx = sample_idx
            end_idx = sample_idx + BATCH_SIZE
            
            f['y_received'][start_idx:end_idx] = y_np
            f['h_channel'][start_idx:end_idx] = h_np
            f['bits'][start_idx:end_idx] = bits_np
            f['snr_db'][start_idx:end_idx] = snr_db
            
            sample_idx += BATCH_SIZE
            pbar.update(1)
            
            # Periodic status update
            if (batch_idx + 1) % 100 == 0:
                elapsed = time.time() - start_time
                samples_generated = sample_idx
                total_samples_target = total_samples * NUM_SNR_POINTS
                progress = samples_generated / total_samples_target
                eta = (elapsed / progress - elapsed) if progress > 0 else 0
                
                pbar.set_postfix({
                    'Samples': f'{samples_generated:,}/{total_samples_target:,}',
                    'ETA': f'{eta/60:.1f} min'
                })
        
        pbar.close()

total_time = time.time() - start_time

print(f"\n✅ Dataset generation complete!")
print(f"   Total time: {total_time/60:.1f} minutes")
print(f"   Samples generated: {sample_idx:,}")
print(f"   File size: {os.path.getsize(dataset_path) / 1024**3:.2f} GB")
print(f"   Throughput: {sample_idx / total_time:.1f} samples/sec")

## 5. Verify Dataset Integrity

In [ ]:
# Load and verify
with h5py.File(dataset_path, 'r') as f:
    print("\n📂 Dataset Contents:")
    print(f"  y_received: {f['y_received'].shape}, dtype={f['y_received'].dtype}")
    print(f"  h_channel: {f['h_channel'].shape}, dtype={f['h_channel'].dtype}")
    print(f"  bits: {f['bits'].shape}, dtype={f['bits'].dtype}")
    print(f"  snr_db: {f['snr_db'].shape}, dtype={f['snr_db'].dtype}")
    
    print("\n📋 Metadata:")
    for key, value in f.attrs.items():
        print(f"  {key}: {value}")
    
    # Sample statistics
    y_sample = f['y_received'][0:1000]
    print(f"\n📊 Sample Statistics (first 1000 samples):")
    print(f"  Mean power: {np.mean(np.abs(y_sample)**2):.4f}")
    print(f"  Std power: {np.std(np.abs(y_sample)**2):.4f}")
    
    # SNR distribution
    snr_values = f['snr_db'][:]
    unique_snrs, counts = np.unique(snr_values, return_counts=True)
    print(f"\n📡 SNR Distribution:")
    for snr, count in zip(unique_snrs, counts):
        print(f"  {snr:.1f} dB: {count:,} samples")

print("\n✅ Dataset verification complete!")

## 6. Visualize Dataset Samples

In [ ]:
# Visualize samples at different SNRs
with h5py.File(dataset_path, 'r') as f:
    fig, axes = plt.subplots(3, 3, figsize=(15, 12))
    
    snr_samples = [-10, -5, 0, 5, 10]  # Representative SNR points
    
    for idx, snr_db in enumerate(snr_samples[:3]):
        # Find sample with this SNR
        snr_mask = np.abs(f['snr_db'][:] - snr_db) < 0.1
        sample_indices = np.where(snr_mask)[0]
        
        if len(sample_indices) > 0:
            sample_idx = sample_indices[0]
            
            # Received signal
            y = f['y_received'][sample_idx, 0]  # First RX antenna
            h = f['h_channel'][sample_idx, 0, 0]  # First RX-TX pair
            
            # Plot received signal
            im1 = axes[idx, 0].imshow(np.abs(y)**2, aspect='auto', cmap='viridis', origin='lower')
            axes[idx, 0].set_title(f'Received Power (SNR={snr_db} dB)')
            axes[idx, 0].set_xlabel('OFDM Symbol')
            axes[idx, 0].set_ylabel('Subcarrier')
            plt.colorbar(im1, ax=axes[idx, 0])
            
            # Plot channel
            im2 = axes[idx, 1].imshow(np.abs(h)**2, aspect='auto', cmap='plasma', origin='lower')
            axes[idx, 1].set_title(f'Channel |H|² (SNR={snr_db} dB)')
            axes[idx, 1].set_xlabel('OFDM Symbol')
            axes[idx, 1].set_ylabel('Subcarrier')
            plt.colorbar(im2, ax=axes[idx, 1])
            
            # Plot constellation (first symbol)
            y_flat = y[:, 0].flatten()  # First OFDM symbol
            axes[idx, 2].scatter(y_flat.real, y_flat.imag, alpha=0.1, s=1)
            axes[idx, 2].set_title(f'Constellation (SNR={snr_db} dB)')
            axes[idx, 2].set_xlabel('In-Phase')
            axes[idx, 2].set_ylabel('Quadrature')
            axes[idx, 2].grid(True, alpha=0.3)
            axes[idx, 2].axis('equal')
    
    plt.tight_layout()
    plt.savefig('/opt/app-root/src/results/dataset_samples.png', dpi=150, bbox_inches='tight')
    plt.show()

print("\n✅ Visualization complete!")

## Summary

**✅ Dataset generation complete!**

You now have:
- **100K PUSCH slots** across 21 SNR points (-10 to +10 dB)
- **~10GB HDF5 file** with compressed storage
- **4 RX antennas** for spatial diversity
- **CDL-C channel model** (Urban Macro 3GPP)
- **16QAM modulation** (MCS-7)

**Next Steps:**
1. Proceed to `02-train-neural-rx.ipynb` to train the neural receiver
2. Expected training time: ~2 hours on RTX 4090
3. Target: 2-3 dB SNR gain at BLER = 10^-2

**Dataset Location:**
```
/opt/app-root/src/data/pusch_dataset.h5
```